In [ ]:
import cobra
import pandas as pd 


Modelnew = cobra.io.read_sbml_model("C:/Users/sofie/OneDrive/Documents/Master - Data/ModelSuc.sbml")    #model with new BOF
ModelOg = cobra.io.read_sbml_model("C:/Users/sofie/OneDrive/Documents/Master - Data/OGModelSuc.sbml") 

In [ ]:
from cobra.flux_analysis import pfba
#parsimonious FBA:
newFBA = pfba(Modelnew)
#print(newFBA.fluxes)

flux_dataframe_new = pd.DataFrame(columns=["Reaction", "Flux"])
for reaction in Modelnew.reactions:
    flux_dataframe_new = flux_dataframe_new.append({"Reaction": reaction.id, "Flux": newFBA.fluxes[reaction.id]}, ignore_index=True)

In [ ]:
#parsimonious FBA:

ogFBA = pfba(ModelOg)
#print(newFBA.fluxes)

flux_dataframe_og = pd.DataFrame(columns=["Reaction", "Flux"])
for reaction in ModelOg.reactions:
    flux_dataframe_og = flux_dataframe_og.append({"Reaction": reaction.id, "Flux": ogFBA.fluxes[reaction.id]}, ignore_index=True)

In [ ]:
#Checking to see if i can remove the biomass synthesis reactions so that i can make a plot with the FVA results.
Reactions_remove_new = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

# Print flux values of reactions to be removed from flux_dataframe_new
print("Flux values to be removed from flux_dataframe_new:")
for index, row in flux_dataframe_new.iterrows():
    if row['Reaction'] in Reactions_remove_new:
        print(f"Reaction: {row['Reaction']}, Flux: {row['Flux']}")


# Iterate through the DataFrame
for index, row in flux_dataframe_new.iterrows():
    if row['Reaction'] in Reactions_remove_new:
        flux_dataframe_new.drop(index, inplace=True)

# Print the updated DataFrame
#print(flux_dataframe_new)
#print(len(flux_dataframe_new))

Reactions_remove_og = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006"]

# Print flux values of reactions to be removed from flux_dataframe_og
print("Flux values to be removed from flux_dataframe_og:")
for index, row in flux_dataframe_og.iterrows():
    if row['Reaction'] in Reactions_remove_og:
        print(f"Reaction: {row['Reaction']}, Flux: {row['Flux']}")

for index, row in flux_dataframe_og.iterrows():
    if row['Reaction'] in Reactions_remove_og:
        flux_dataframe_og.drop(index, inplace=True)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


#Colors = ['#1f78b4', '#b2df8a', '#33a02c', '#cab2d6', '#a2438d']


FBAnew = flux_dataframe_new["Flux"].tolist()
FBAog  = flux_dataframe_og["Flux"].tolist()
reactions = range(1, len(FBAnew) + 1)

plt.figure(figsize=(5, 6)) 

# Plot the range values for model 1
plt.scatter(reactions, FBAnew, color='#a2438d', label='new BOF', alpha = 1)

# Plot the range values for model 2
plt.scatter(reactions, FBAog, color='gray', label='Original BOF', alpha= 0.4)
#plt.gca().set_facecolor('white')

# Set labels for the range subplot
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.ylabel("  ", fontsize = 12) #Reaction flux (mmol gDW$^{-1}$ h$^{-1}$)
plt.xlabel("Reaction number", fontsize = 12) #Reaction number
plt.ylim(-4.9, 5.0)


plt.legend(fontsize = 10, loc="upper right")
plt.savefig('C:/Users/sofie/OneDrive/Documents/Master - Data/Figurer/FBA_FVA/FBA--Succinate.svg', dpi=300,  bbox_inches='tight')
plt.show()

#enzymbegrenset modell vil ta kostnad for reaksjonsvei i betraktning.
    

In [ ]:

sorted_dataframe_new = flux_dataframe_new.sort_values(by="Flux", ascending=False)
print(sorted_dataframe_new[0:50])

In [ ]:
sorted_dataframe_og = flux_dataframe_og.sort_values(by="Flux", ascending=False)
print(sorted_dataframe_og [0:50])

In [ ]:
Threshold = 0.1


# Merge the dataframes based on reaction IDs
merged_df = pd.merge(flux_dataframe_new, flux_dataframe_og, on='Reaction', suffixes=['_new', '_old'])

# Filter out rows where flux values differ
differing_flux = merged_df[abs(merged_df['Flux_new'] - merged_df['Flux_old']) > Threshold]

# Print reaction IDs and differing flux values
for index, row in differing_flux.iterrows():
    reaction_id = row['Reaction']
    flux_new = row['Flux_new']
    flux_old = row['Flux_old']
    print(f"Reaction ID: {reaction_id}, Flux (new): {flux_new}, Flux (old): {flux_old}")

